In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import os.path
import matplotlib.pyplot as plt
import random
import math

from IPython.core.debugger import set_trace

In [ ]:
golden_run_path = "cg/cg_in8/golden.log"
golden_run = pd.read_csv(golden_run_path,  sep=',', names=['file', 'linenum', 'variable', 'value'], header=0)
golden_run_value = np.array(golden_run.value, dtype='float')
cg_fault_injection = pd.read_csv('cg_in8.csv')
SIZE = len(cg_fault_injection)
print(SIZE)

In [ ]:
cg_fault_injection.outcome

# Experiment 1 -- CG
Take all the fault injections in the inital condition. Find all the fault injection cases that result into masked. For each masked case, compare error run with the golden run to get a error run curve. Combine all the error run curve to construct a error boundary for masked and SDC.

In [ ]:
cg_fault_injection_experiment = pd.read_csv('matrix/in10_data/in10/injectlog.log',  sep=' ', names=['file', 'linenum', 'variable','byte_num', 'corrupted','init_value','to', 'corrupt_value', 'mask', 'byte','expo', 'ss', 'op', 'diffnorm', 'empty'])

### SDC Ratio over entire program

In [ ]:
iters = list(cg_fault_injection.iter)
iters.reverse()
count  = 0
last_zero_iteration_index = SIZE - iters.index(0)
dynamic_step_to_record_computation_result = int(last_zero_iteration_index/64)

for i in range(last_zero_iteration_index, len(iters)):
    if cg_fault_injection.outcome[i] == "SDC":
        count += 1

print("SDC ratio over entire program: ", count/len(iters))

In [ ]:
#only tracking the maximum value of the masked run after this time step.
tracking_length = len(golden_run_value)

def construct_boundary(number_of_dynamic_instruction):
    boundary = []
    ground_truth = {"Masked": 0, "SDC":0, "DUE":0}

    for i in range(tracking_length):
        boundary.append({'max':0, 'min':0})

    for i in range(number_of_dynamic_instruction):
        index = int(random.random() * SIZE)
        
        ground_truth[cg_fault_injection.outcome[index]] += 1
        
        if cg_fault_injection.outcome[index] == "Masked":
            masked_run_path = "cg_simulation/appstate_"+ str(index)+".log" 

            #Check whether the tracking file is on the path
            #TODO: check whether can regenerate the data
            if not os.path.isfile(masked_run_path):
                continue

            masked_run = pd.read_csv(masked_run_path,  sep=",", names=['file', 'linenum', 'variable', 'value'], header=0)
            masked_run_value = np.array(masked_run.value, dtype='float')
            
            #This is a werid information in the data.
            #TODO: may check why such outcome is shown
            if len(masked_run_value) < tracking_length:
                print("odd!")
                continue
            
            #array contain NAN, ignore the run
            if np.isnan(np.min(masked_run_value)):
                continue
            
            comparision_result = masked_run_value[0:tracking_length] - golden_run_value
        else:
            continue
        
        
        for j in range(dynamic_step_to_record_computation_result, tracking_length):
            if comparision_result[j] > boundary[j]['max'] and comparision_result[j] >= 0:
                boundary[j]['max'] = comparision_result[j]

            if comparision_result[j] < boundary[j]['min'] and comparision_result[j] < 0:
                boundary[j]['min'] = comparision_result[j]
                
    return [boundary, ground_truth]

def predict(boundary):
    masked_true_positive = 0
    masked_false_positive = 0
    sdc_true_positive = 0
    sdc_false_positive = 0
    result = {}

    for i in range(last_zero_iteration_index + 1, SIZE):
        index = math.floor(i/64)
        if cg_fault_injection.out_xor[i] < boundary[index]['max'] and cg_fault_injection.out_xor[i] > boundary[index]['min']:
            if cg_fault_injection.outcome[i] == "Masked":
                masked_true_positive += 1
            else:
                masked_false_positive += 1
        else:
            if cg_fault_injection.outcome[i] == "SDC":
                sdc_true_positive += 1
            else:
                sdc_false_positive += 1

    result["predict masked case"] =  (masked_true_positive + masked_false_positive)/SIZE
    result["predict SDC case"] = (sdc_true_positive + sdc_false_positive)/SIZE
    result["masked prediction accuracy"] = masked_true_positive/(masked_true_positive + masked_false_positive)
    result["SDC prediction accuracy"] = sdc_true_positive/(sdc_true_positive + sdc_false_positive)
    
    return result

## Can we think this as a lable propagation problem?

In [ ]:
experiments = []

ground_truth = []


for i in range(10):
    print(i)
    boundary = construct_boundary(2000)
    
    experiments.append(predict(boundary[0]))
    ground_truth.append(boundary[1])

    
    
uniform_test = []
intuition = []

for i in range(10):
    intuition.append(experiments[i]['predict SDC case'])
    uniform_test.append(ground_truth[i]['SDC']/2000)

print(np.mean(intuition))
print(intuition)
print(np.mean(uniform_test))

# Experiment 2 -- FFT

In [2]:
path = "fft/fft_complete/"

golden_run_path = path+"golden.log"
golden_run = pd.read_csv(golden_run_path,  sep=' ', names=['file', 'linenum', 'variable', 'value'])
golden_run_value = np.array(golden_run.value, dtype='float')
fault_injection_experiment = pd.read_csv(path+'injectlog.log',  sep=' ', names=['file', 'linenum', 'variable','byte_num', 'corrupted','init_value','to', 'corrupt_value', 'mask', 'byte','expo', 'ss', 'op', 'diffnorm', 'empty'])
SIZE = len(fault_injection_experiment)

In [17]:
def load_data(path):
    
    golden_run_path = path + "golden.log"
    golden_run = pd.read_csv(golden_run_path,  sep=' ', names=['file', 'linenum', 'variable', 'value'])
    golden_run_value = np.array(golden_run.value, dtype='float')
    fault_injection_experiment = pd.read_csv(path+"injectlog.log",  sep=' ', names=['file', 'linenum', 'variable','byte_num', 'corrupted','init_value','to', 'corrupt_value', 'mask', 'byte','expo', 'ss', 'op', 'diffnorm', 'empty'])
    
    return [golden_run,fault_injection_experiment]

def SDC_ratio(experiments): 
    sdc_count = 0
    for i in range(len(experiments)):
        if float(np.array(experiments.diffnorm)[i]) > 0.07:
            sdc_count += 1
    
    return sdc_count / len(experiments)

def getBoundary(golden_run, percent, experiments, path):
    boundary = []
    nums = int(len(experiments) * percent)

    for i in range(len(golden_run)):
        boundary.append({"min": 0, "max":0})

    for i in range(nums):
        if float(np.array(experiments.diffnorm)[i]) > 0.07:
            continue
    
        index = int(random.random() * len(experiments))

        file_path = path+"appstate_"+str(index)+".log"
    
        fault_inject_run = pd.read_csv(file_path,  sep=' ', names=["file", "linenum", "variable", "value"])
    
        if len(fault_inject_run) < len(golden_run):
            print("weird!")
            continue
    
        values = np.array(fault_inject_run.value[0:len(golden_run)], dtype="float") - np.array(golden_run.value, dtype="float")
    
        for j in range(len(golden_run)):
            if values[j] >= 0 and values[j] > boundary[j]["max"]:
                boundary[j]["max"] = values[j]
                       
            if values[j] < 0 and values[j] < boundary[j]["min"]:
                boundary[j]["min"] = values[j] 
    
    return boundary

def prediction(experiments, boundary):
    positive_prediction = 0
    negative_prediction = 0
    
    start_index = int(len(experiments.diffnorm)*0.4)
    end_index = len(experiments.diffnorm)
    
    for i in range(start_index, end_index):
        index = math.floor(i/64)
        inject_error = float(np.array(experiments.corrupt_value)[i]) - float(np.array(experiments.init_value)[i])
    
        if(inject_error > boundary[index]["min"] and inject_error < boundary[index]["max"]):
            if float(np.array(experiments.diffnorm)[i]) < 0.07:
                positive_prediction += 1
            else:
                negative_prediction += 1
    
    return {"positive": positive_prediction/len(experiments), "negative": negative_prediction/len(experiments)}

# Predict Result of 20 x 20 Matrix

In [5]:
golden_run, experiments = load_data(path)
boundary = getBoundary(golden_run, 0.01, experiments, path)
experiments = pd.read_csv("fft/fft_injectlog.log",  sep=' ', names=['file', 'linenum', 'variable','byte_num', 'corrupted','init_value','to', 'corrupt_value', 'mask', 'byte','expo', 'ss', 'op', 'diffnorm', 'empty'])

print(len(experiments))
print(SDC_ratio(experiments))
#print(prediction(experiments, boundary))

C:\Users\zhimin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2163264


In [ ]:
print(SDC_ratio(experiments))
#np.array(experiments.diffnorm)[641:700]


In [ ]:
print(SDC_ratio(experiments))

# Using 6400 fault injections run in FFT to test the masked case. 

In [ ]:
boundary = []

for i in range(len(golden_run)):
    boundary.append({"min": 0, "max":0})

for i in range(8400):
    
    if fault_injection_experiment.diffnorm[i] > 0.07:
        continue
    
    index = int(random.random() * len(fault_injection_experiment))

    file_path = "matrix/in27_data/in27/appstate_"+str(index)+".log"
    
    fault_inject_run = pd.read_csv(file_path,  sep=' ', names=["file", "linenum", "variable", "value"])
    
    if len(fault_inject_run) < len(golden_run):
        print("weird!")
        continue
    
    values = np.array(fault_inject_run.value[0:len(golden_run)], dtype="float") - np.array(golden_run.value, dtype="float")
    
    for j in range(len(golden_run)):
        if values[j] >= 0 and values[j] > boundary[j]["max"]:
            boundary[j]["max"] = values[j]
                       
        if values[j] < 0 and values[j] < boundary[j]["min"]:
            boundary[j]["min"] = values[j] 

In [ ]:
positive_prediction = 0
negative_prediction = 0
for i in range(len(fault_injection_experiment.diffnorm)):
    
    index = math.floor(i/64)
    
    #if i%64 <= 52:
    #    continue
    
    inject_error = float(fault_injection_experiment.corrupt_value[i]) - float(fault_injection_experiment.init_value[i])
    
    if(inject_error > boundary[index]["min"] and inject_error < boundary[index]["max"]):
        if fault_injection_experiment.diffnorm[i] < 0.07:
            positive_prediction += 1
        else:
            negative_prediction += 1

In [ ]:
#relative error and absolute error
#for i in range(len(golden_run_value)):
#   if golden_run_value[i] != 0:
#        absolute = abs(golden_run_value[i] -  masked_run_value[i])

false_positive = 0
true_positive = 0
unsure = 0
total = (811 - 160) * 64
for i in range(160, 811):
    diff = abs(golden_run_value[i] -  masked_run_value[i]) 
    for b in range(64):
        if diff > abs(cg_fault_injection.out_xor[i * 64 + b]) and cg_fault_injection.outcome[i * 64 + b] != "Masked":
            #print(cg_fault_injection.diffnormr[i * 64 + b], cg_fault_injection.Variable[i * 64 + b], (diff - abs(cg_fault_injection.out_xor[i * 64 + b])))
            false_positive += 1
            print(i * 64 + b)
        elif diff > abs(cg_fault_injection.out_xor[i * 64 + b]) and cg_fault_injection.outcome[i * 64 + b] == "Masked":
            true_positive += 1
        else:
            unsure += 1
    
    #sdc_diff = abs(golden_run_value[i] -  sdc_run_value[i])
    #for b in range(64):
    #    if sdc_diff < abs(cg_fault_injection.out_xor[i * 64 + b]) and cg_fault_injection.outcome[i * 64 + b] != "SDC":
    #        false_positive += 1
    #    elif sdc_diff < abs(cg_fault_injection.out_xor[i * 64 + b]) and cg_fault_injection.outcome[i * 64 + b] == "SDC":
    #        true_positive += 1
    #    else:
    #        unsure +=1
            
print(false_positive/total)
print(true_positive/total)
print(unsure/total)
    
#print(i, cg_fault_injection.Variable[i], cg_fault_injection.out_xor[i], cg_fault_injection.outcome[i])
#for i in range(243*64,244*64):
#    print(i,cg_fault_injection.Variable[i], cg_fault_injection.out_xor[i], cg_fault_injection.outcome[i])
#print(golden_run_value[243])
#print(len(cg_fault_injection))
#golden_run_value-error_run_value[0:811]

In [ ]:
number_of_instructions = {}

propagation_path = []

for index in range(100):
    if index in[15731, 16246, 17523]:
        continue
    
    file_path = "cg_simulation/appstate_"+str(index)+".log"
    error_run = pd.read_csv(file_path,  sep=' ')
    
    
    #For different fault injection case, what is the number of different execution dynamic instructions
    if len(error_run) in number_of_instructions:
        number_of_instructions[len(error_run)] += 1
    else:
        number_of_instructions[len(error_run)] = 1
        
    #
    index = min(len(golden_run), len(error_run))
    #result = np.array(golden_run.value, dtype=float)[:index] - np.array(error_run.value, dtype=float)[:index]
    #propagation_elements = golden_run.line[np.where(result != 0)[0]]
    
    #path = "=>"
    #number_of_propagation_element = 5
    #for e in propagation_elements:
    #    if str(e) not in path:
    #        path += str(e)
    #        path += "=>"
    #        number_of_propagation_element -= 1
        
    #    if number_of_propagation_element == 0:
    #        break
    #if path not in propagation_path:
    #    propagation_path.append(path)

## What is the probability of the different number of storing dynamic instructions.
## What is the probability of SDC.

It's expected that if the program ends early, then there an unexpected crash during the program execution in the earlier stage. At the same time, if the program has more execution than it expected. It will have high chance causes silent data corruption.

In [ ]:
cg_outcome = cg_fault_injection['outcome'].value_counts()

print(cg_outcome)
print(sum(cg_outcome))
#propagation_path

In [ ]:
number_of_instructions

# If you want to rewrite your code.

4. interactive fault injection campaign.

### 1. A sensitive analysis across the whole program with a mapping framework.
A desity scatter plot to understand the input and output sensitivity of the program. User can selective choose the high sensitive data and mapping back to the original visualization.
    
    a. how many clusters in the plot.
    
    b. Where is each of them comes from. 
    
    c. The sample that comes from the fault injection in same location.

#### 2. Explore function level resiliency of the program. What's the difference compare to the source code level?

    a.Explore the resiliency of different program component. Aggregate the data in variable level


#### 3. How to measure the impact of one variable/function over the other?
 


# The number of times a function is called.

In [ ]:
counter = {}

line_to_func = {167:'readA', 175:'readB', 33:'waxpby', 25:'matvec', 48:'dot_r2', 75:'solve_cg', 76:'solve_cg',87:'solve_cg',57:'dot', 90:'solve_cg',91:'solve_cg',40:"daxpby", 82:'daxpby',83:'daxpby',84:'daxpby'}

for index, row in golden_run.iterrows():
    key = line_to_func[row['line']]
    if key not in counter:
        counter[key] = 1
    else:
        counter[key] += 1

In [ ]:
number_of_instruction = sum(list(counter.values()))
for item in counter.items():
    print(item[0], item[1]/number_of_instruction * 100)

# Error Propagation Analysis in one program component.

In [ ]:
golden_run_path = "cg_simulation/golden.log"
golden_run = pd.read_csv(golden_run_path,  sep=' ', names=['file', 'linenum', 'variable', 'value'])
golden_run_value = np.array(golden_run.value)
##### this is a very interesting but werid case
##### There is a NAN occur during the computation, but the  error is masked at the end.
#index_range = [17341, 24381]

##### cg p_ap_dot first interation 64 experiment
#index_range = [15552, 15603]

#file_path = "cg_simulation1/appstate_"+str(17341)+".log"
#error_run = pd.read_csv(file_path,  sep=' ')

#for i in range(len(error_run.value)):
#    print(error_run.line[i], error_run.variable[i], error_run.value[i])
#print(len(error_run.value))

#print(error_run)
#data_set = {}


#golden_run_value = np.array(golden_run.value[0:800], dtype='float')

#for index in range(index_range[0], index_range[1]+1):
#    file_path = "cg_simulation/appstate_"+str(index)+".log"
#    error_run = pd.read_csv(file_path,  sep=' ', names=['file', 'linenum', 'variable', 'value'])

#    data_set[index] = golden_run_value - np.array(error_run.value[0:800], dtype='float')


##### a specific case line 75 case 10290
##### a specific case line 75 case 10296
indexs =  [10290, 10274]# 10288] #10297]# 10296, 10289]

delta_x = [65536, 1.0]# 1321922331132047.5]#, -307779.3308780107]
x_data = range(0, 811, 1)

for index,item in enumerate(indexs): 
    file_path = "cg_simulation/appstate_"+str(item)+".log"
    error_run = pd.read_csv(file_path,  sep=' ', names=['file', 'linenum', 'variable', 'value'])
    error_run_value = np.array(error_run.value)

    sensitivity_val = (error_run_value[0:811] - golden_run_value[0:811])/delta_x[index]
    sns.lineplot(x=x_data, y=sensitivity_val, sort=False, lw=1)
    
print(sensitivity_val.argmax())

In [ ]:
#for index in range(index_range[0], index_range[1]+1):
#    print(len(data_set[index].values))
#data_set[index] = golden_run.values - error_run.values   


x_data = range(0,811,1)
sns.lineplot(x=x_data, y=sensitivity_val, sort=False, lw=1)

#for index in range(index_range[0], index_range[0]+20): 
#    sns.lineplot(x=x_data, y=data_set[index], sort=False, lw=1)

## What role the truncation error is playing in the propagation analysis
## What's the threshold of different program components.
## How much error is masked by the truncate error and How much error is masked by the model feature